In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
import numpy as np

In [15]:
import os

base_path = "/content/drive/My Drive/Nigerian_Market_Data"

# Check if the directory exists
if os.path.exists(base_path):
    print(f"Contents of {base_path}:")
    for item in os.listdir(base_path):
        print(item)
else:
    print(f"The directory {base_path} does not exist. Please check the path and ensure Google Drive is mounted correctly.")

# Original code (commented out for diagnosis)
prices = pd.read_csv(f"{base_path}/nigerian_market_prices.csv.csv")
transactions = pd.read_csv(f"{base_path}/nigerian_market_transactions_assignment.csv")

Contents of /content/drive/My Drive/Nigerian_Market_Data:
nigerian_market_prices.csv.csv
nigerian_market_transactions_assignment.csv


In [14]:
prices.head()

NameError: name 'prices' is not defined

In [16]:
display(prices.head())

,market_id,market_name,state,product,price,unit,date,trader_name
0,MKT001,Bodija Market,Oyo,Beans,"₦5,770",Kg,01-02-2024,mama nkechi
1,MKT003,Ogbete Market,Enugu,Yam,"₦2,534",basket,07/02/2024,bello
2,MKT001,Bodija Market,Oyo,Yam,NaN,50kg,22/02/2024,alhaji musa
3,MKT004,Wuse Market,Abuja,Tomatoes,NaN,Paint Rubber,29-01-2024,mama nkechi
4,MKT003,Ogbete Market,Enugu,Gari,-2616,kg,2024/01/31,sadiya


In [17]:
transactions.head()

,transaction_id,market_id,trader_id,trader_name,product,quantity,revenue,payment_method,transaction_date
0,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53
1,TXN0002,MKT004,TR001,SADIYA,Beans,14,492226,POS,2024-01-05 13:46
2,TXN0003,MKT002,TR006,ALHAJI MUSA,Tomatoes,8,283392,Transfer,2024-01-07 16:53
3,TXN0004,MKT003,TR001,Sadiya,Beans,13,248027,Transfer,2024-03-02 10:14
4,TXN0005,MKT005,TR002,Alhaji Musa,Rice,6,46302,Cash,2024-01-27 14:37


In [18]:
prices.info()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   market_id    1000 non-null   object
 1   market_name  1000 non-null   object
 2   state        1000 non-null   object
 3   product      1000 non-null   object
 4   price        925 non-null    object
 5   unit         1000 non-null   object
 6   date         1000 non-null   object
 7   trader_name  1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    1000 non-null   object
 1   market_id         1000 non-null   object
 2   trader_id         1000 non-null   object
 3   trader_name       1000 non-null   object
 4   product           1000 non-null   object
 5   quantity    

In [19]:
prices.columns = prices.columns.str.lower().str.strip().str.replace(" ", "_")
transactions.columns = transactions.columns.str.lower().str.strip().str.replace(" ", "_")

In [20]:
prices = prices.drop_duplicates()
transactions = transactions.drop_duplicates()

In [21]:
prices.isna().sum()

,0
market_id,0
market_name,0
state,0
product,0
price,75
unit,0
date,0
trader_name,0


In [22]:
transactions.isna().sum()

,0
transaction_id,0
market_id,0
trader_id,0
trader_name,0
product,0
quantity,0
revenue,0
payment_method,0
transaction_date,0


In [24]:
if "price" in prices.columns:
    # Clean the 'price' column: remove currency symbol and commas, then convert to numeric
    prices["price"] = prices["price"].astype(str).str.replace('₦', '', regex=False).str.replace(',', '', regex=False)
    # Convert to numeric, coercing errors to NaN for values that can't be converted
    prices["price"] = pd.to_numeric(prices["price"], errors='coerce')
    # Fill NaN values with the median of the now numeric 'price' column
    prices["price"] = prices["price"].fillna(prices["price"].median())

In [26]:
transactions = transactions.dropna(subset=["market_id", "product"])

In [27]:
cleaned_data = pd.merge(
    transactions,
    prices,
    on=["market_id", "product"],
    how="left"
)

In [28]:
cleaned_data.head()

,transaction_id,market_id,trader_id,trader_name_x,product,quantity,revenue,payment_method,transaction_date,market_name,state,price,unit,date,trader_name_y
0,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53,Ariaria Market,Abia,2976.0,basket,01-03-2024,BELLO
1,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53,Ariaria Market,Abia,4916.0,Basket,2024/01/11,bello
2,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53,Ariaria Market,Abia,4508.0,50kg,10-02-2024,bello
3,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53,Ariaria Market,Abia,1035.0,50kg,24/03/2024,sadiya
4,TXN0001,MKT005,TR005,ALHAJI MUSA,Gari,8,86160,Transfer,2024-02-03 13:53,Ariaria Market,Abia,3293.0,basket,08-01-2024,sadiya


In [29]:
cleaned_data.shape

(40185, 15)

In [30]:
cleaned_data.isna().sum()

,0
transaction_id,0
market_id,0
trader_id,0
trader_name_x,0
product,0
quantity,0
revenue,0
payment_method,0
transaction_date,0
market_name,0


In [31]:
if "price" in cleaned_data.columns:
    cleaned_data["price"] = cleaned_data["price"].fillna(cleaned_data["price"].median())

In [32]:
output_path = f"{base_path}/cleaned_market_data.csv"

cleaned_data.to_csv(output_path, index=False)

output_path

'/content/drive/My Drive/Nigerian_Market_Data/cleaned_market_data.csv'

In [33]:
from google.colab import files

files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>